In [1152]:
import json
import jsonlines
from json_logic import jsonLogic
from json_logic.builtins import BUILTINS, op_var, op_log
from functools import reduce

In [1153]:
def courses(data=None, *_ignored):
    return list(filter(None, op_var(data, "courses")))


def course(data=None, code="", number="", *_ignored):
    for course in courses(data):
        if course["code"] == code and str(course["number"]).startswith(number):
            return course
    return None


def units(data=None, *courses):
    flatten_list = lambda *lists: reduce(lambda a, b: a + b, lists, [])
    courses = flatten_list(*courses)
    
    total = 0
    for course in filter(None, courses):
        total += course["units"]
    return total

def consent(data=None, *consenter):
    op_log(data, f"Consent of: {', '.join(consenter) if consenter else 'None'}")
    return True

def program(data=None, *conditioned_programs):
    programs = op_var(data, "programs")
    for p in programs:
        components = p["components"]
        if all(c in components for c in conditioned_programs):
            return p
    return None

ops = {
    **BUILTINS,
    "courses": courses,
    "course": course,
    "units": units,
    "consent": consent,
    "program": program,
}

# Examples


In [1154]:
NUMBER_OF_EXAMPLES = 10

ors = [None] * NUMBER_OF_EXAMPLES
rules = [None] * NUMBER_OF_EXAMPLES
datas = [None] * NUMBER_OF_EXAMPLES

## Example 1

In [1155]:
ors[0] = {
    "cid": "161622",
    "code": "CPSC",
    "number": "530",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "CPSC 351; or 3 units from CPSC 219, 233 or 235 and 3 units from MATH 271, 273, 315 and 3 units from STAT 205, 213, 321.",
    "antireq": "Credit for CPSC 530 and 630 will not be allowed.",
    "coreq": None,
}

In [1156]:
rules[0] = {
    "cid": "161622",
    "prereq": {
        "or": [
            {"course": ["CPSC", "351"]},
            {
                "and": [
                    {
                        ">=": [
                            {
                                "units": [
                                    {"course": ["CPSC", "219"]},
                                    {"course": ["CPSC", "233"]},
                                    {"course": ["CPSC", "235"]},
                                ]
                            },
                            3,
                        ]
                    },
                    {
                        ">=": [
                            {
                                "units": [
                                    {"course": ["MATH", "271"]},
                                    {"course": ["MATH", "273"]},
                                    {"course": ["MATH", "315"]},
                                ]
                            },
                            3,
                        ]
                    },
                    {
                        ">=": [
                            {
                                "units": [
                                    {"course": ["STAT", "205"]},
                                    {"course": ["STAT", "213"]},
                                    {"course": ["STAT", "321"]},
                                ]
                            },
                            3,
                        ]
                    },
                ]
            },
        ]
    },
    "antireq": {
        "or": [
            {"course": ["CPSC", "530"]},
            {"course": ["CPSC", "630"]},
        ]
    },
    "coreq": None,
}

In [1157]:
datas[0] = {
    "courses": [
        {"code": "CPSC", "number": "329", "gpv": 2.7, "semester": "W2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "526", "gpv": 2.7, "semester": "F2023", "units": 6.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "219", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "MATH", "number": "313", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "MATH", "number": "271", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "STAT", "number": "205", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "351", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": []
}

a = jsonLogic(rules[0]["prereq"], datas[0], ops)
b = jsonLogic(rules[0]["antireq"], datas[0], ops)

bool(a), bool(b)

(True, False)

## Example 2

In [1158]:
ors[1] = {
    "cid": "160417",
    "code": "CPSC",
    "number": "219",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "CPSC 217 or DATA 211.",
    "antireq": "Credit for CPSC 219 and any of 233, 235, ENEL 497 or ENCM 493 will not be allowed.",
    "coreq": None,
}

In [1159]:
rules[1] = {
    "cid": "160417",
    "prereq": {
        "or": [
            {"course": ["CPSC", "217"]}, 
            {"course": ["DATA", "211"]}
        ]
    },
    "antireq": {
        "or": [
            [
                {"course": ["CPSC", "219"]},
                {"course": ["CPSC", "233"]},
                {"course": ["CPSC", "235"]},
                {"course": ["ENEL", "497"]},
                {"course": ["ENCM", "493"]},
            ],
        ]
    },
    "coreq": None,
}

In [1160]:
datas[1] = {
    "courses": [
        {"code": "CPSC", "number": "217", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "219", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "233", "gpv": 2.7, "semester": "F2023", "units": 6.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": []
}

a = jsonLogic(rules[1]["prereq"], datas[1], operations=ops)
b = jsonLogic(rules[1]["antireq"], datas[1], operations=ops)

bool(a), bool(b)

(True, True)

## Example 3

In [1161]:
ors[2] = {
    "cid": "164005",
    "code": "CPSC",
    "number": "405",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "SENG 300 or 301.",
    "antireq": "Credit for CPSC 405 and ENTI 415 will not be allowed.",
    "coreq": None,
}

In [1162]:
rules[2] = {
    "cid": "164005",
    "prereq": {
        "or": [
            {"course": ["SENG", "300"]}, 
            {"course": ["SENG", "301"]}
        ]
    },
    "antireq": {
        "or": [
            {"course": ["CPSC", "405"]},
            {"course": ["ENTI", "415"]},
        ]
    },
    "coreq": None,
}

In [1163]:
datas[2] = {
    "courses": [
        {"code": "SENG", "number": "300", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "405", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": []
}

a = jsonLogic(rules[2]["prereq"], datas[2], operations=ops)
b = jsonLogic(rules[2]["antireq"], datas[2], operations=ops)


bool(a), bool(b)

(True, True)

## Example 4

In [1164]:
ors[3] = {
    "cid": "107275",
    "code": "CPSC",
    "number": "550B",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 6.0,
    "prereq": "CPSC 441, 457 and consent of the Department.",
    "antireq": None,
    "coreq": None,
}

In [1165]:
rules[3] = {
    "cid": "107275",
    "prereq": {
        "and": [
            {"course": ["CPSC", "441"]},
            {"course": ["CPSC", "457"]},
            {"consent": ["CPSC"]},
        ]
    },
    "antireq": None,
    "coreq": None,
}

In [1166]:
datas[3] = {
    "courses": [
        {"code": "CPSC", "number": "441", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "457", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": [],
}

a = jsonLogic(rules[3]["prereq"], datas[3], operations=ops)
b = jsonLogic(rules[3]["antireq"], datas[3], operations=ops)


bool(a), bool(b)

"Consent of: CPSC"


(True, False)

## Example 5

In [1167]:
ors[4] = {
    "cid": "161195",
    "code": "CPSC",
    "number": "449",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "CPSC 319 or 331; CPSC 313 or 351; and PHIL 279 or 377.",
    "antireq": "Credit for CPSC 449 and 349 will not be allowed.",
    "coreq": None,
}

In [1168]:
rules[4] = {
    "cid": "161195",
    "prereq": {
        "and": [
            {
                "or": [
                    {"course": ["CPSC", "319"]},
                    {"course": ["CPSC", "331"]},
                ]
            },
            {
                "or": [
                    {"course": ["CPSC", "313"]},
                    {"course": ["CPSC", "351"]},
                ]
            },
            {
                "or": [
                    {"course": ["PHIL", "279"]},
                    {"course": ["PHIL", "377"]},
                ]
            },
        ]
    },
    "antireq": {
        "or": [
            {"course": ["CPSC", "449"]},
            {"course": ["CPSC", "349"]},
        ]
    },
    "coreq": None,
}

In [1169]:
datas[4] = {
    "courses": [
        {"code": "CPSC", "number": "319", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "351", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "PHIL", "number": "279", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": []
}

a = jsonLogic(rules[4]["prereq"], datas[4], operations=ops)
b = jsonLogic(rules[4]["antireq"], datas[4], operations=ops)


bool(a), bool(b)

(True, False)

## Example 6

In [1170]:
ors[5] = {
    "cid": "107229",
    "code": "CPSC",
    "number": "453",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "CPSC 319 or 331; and MATH 211 or 213; and 3 units from MATH 253, 267, 277, 283 or AMAT 219.",
    "antireq": None,
    "coreq": None,
}

In [1171]:
rules[5] = {
    "cid": "107229",
    "prereq": {
        "and": [
            {
                "or": [
                    {"course": ["CPSC", "319"]},
                    {"course": ["CPSC", "331"]},
                ]
            },
            {
                "or": [
                    {"course": ["MATH", "211"]},
                    {"course": ["MATH", "213"]},
                ]
            },
            {
                ">=": [
                    {
                        "units": [
                            {"course": ["MATH", "253"]},
                            {"course": ["MATH", "267"]},
                            {"course": ["MATH", "277"]},
                            {"course": ["MATH", "283"]},
                            {"course": ["AMAT", "219"]},
                        ]
                    },
                    3,
                ]
            },
        ]
    },
    "antireq": None,
    "coreq": None,
}

## Example 7

In [1172]:
ors[6] = {
    "cid": "162552",
    "code": "CPSC",
    "number": "502.07",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 6.0,
    "prereq": "Consent of the Department.",
    "antireq": "Credit for both CPSC 502 and any of 503, ENSF 599 or 591 will not be allowed.",
    "coreq": None,
}

In [1173]:
rules[6] = {
    "cid": "162552",
    "prereq": {
        "consent": ["CPSC"]
    },
    "antireq": {
        "or": [
            {"course": ["CPSC", "502"]},
            {"course": ["CPSC", "503"]},
            {"course": ["ENSF", "599"]},
            {"course": ["ENEL", "591"]},
        ]
    },
    "coreq": None,
}

## Example 8

In [1174]:
ors[7] = {
    "cid": "107154",
    "code": "CPSC",
    "number": "203",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": None,
    "antireq": "Not open for registration to Computer Science majors.",
    "coreq": None,
}

In [1175]:
rules[7] = {
    "cid": "107154",
    "prereq": None,
    "antireq": {
        "program": ["CPSC-MAJ"]
    },
    "coreq": None,
}

In [1176]:
datas[7] = {
    "courses": [
        {"code": "CPSC", "number": "231", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]}
    ],
    "programs": [
        {"code": "CPSCBAH", "components": ["CPSC-MAJ", "CPSC-HON", "SENG-CON"]},
        {"code": "DATA-MIN", "components": ["DATA-MIN"]},
    ],
}

a = jsonLogic(rules[7]["prereq"], datas[7], operations=ops)
b = jsonLogic(rules[7]["antireq"], datas[7], operations=ops)

bool(a), bool(b)

(False, True)

## Example 9

In [1177]:
ors[8] = {
    "cid": "107164",
    "code": "CPSC",
    "number": "231",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "Admission to Computer Science, Bioinformatics, or Natural Science with a primary concentration in Computer Science.",
    "antireq": "Credit for CPSC 231 and any of CPSC 215, 217, 235, DATA 211, ENCM 339, ENGG 233, or ENDG 233 will not be allowed.",
    "coreq": None,
}

In [1178]:
rules[8] = {
    "cid": "107164",
    "prereq": {
        "or": [
            {"program": ["CPSC-MAJ"]},
            {"program": ["CPSC-MIN"]},
            {"program": ["BINF-MAJ"]},
            {"program": ["BINF-MIN"]},
            {"program": ["NTSC-MAJ", "CPSC-CON"]},
        ]
    },
    "antireq": {
        "or": [
            {"course": ["CPSC", "231"]},
            {"course": ["CPSC", "215"]},
            {"course": ["CPSC", "217"]},
            {"course": ["CPSC", "235"]},
            {"course": ["DATA", "211"]},
            {"course": ["ENCM", "339"]},
            {"course": ["ENGG", "233"]},
            {"course": ["ENDG", "233"]},
        ]
    },
    "coreq": None,
}

In [1179]:
datas[8] = {
    "courses": [
        {"code": "CPSC", "number": "231", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]}
    ],
    "programs": [
        # {"code": "CPSCBAH", "components": ["CPSC-MAJ", "CPSC-HON", "SENG-CON"]},
        {"code": "NTSCBAH", "components": ["NTSC-MAJ", "NTSC-HON", "CPSC-CON"]},
        {"code": "DATA-MIN", "components": ["DATA-MIN"]},
    ],
}

a = jsonLogic(rules[8]["prereq"], datas[8], operations=ops)
b = jsonLogic(rules[8]["antireq"], datas[8], operations=ops)

bool(a), bool(b)

(True, True)

## Example 10

In [1180]:
ors[9] = {
    "cid": "107273",
    "code": "CPSC",
    "number": "535",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "One of MATH 311, 313, 353, AMAT 307, 311, PMAT 331.",
    "antireq": "Credit for CPSC 535 and 635 will not be allowed.",
    "coreq": None,
}

In [1181]:
rules[9] = {
    "cid": "107273",
    "prereq": {
        "or": [
            {"course": ["MATH", "311"]},
            {"course": ["MATH", "313"]},
            {"course": ["MATH", "353"]},
            {"course": ["AMAT", "307"]},
            {"course": ["AMAT", "311"]},
            {"course": ["PMAT", "331"]},
        ]
    },
    "antireq": {
        "or": [
            {"course": ["CPSC", "535"]},
            {"course": ["CPSC", "635"]},
        ]
    },
    "coreq": None,
}

# Aggregation

In [1182]:
SYSTEM_MESSAGE = {
    "role": "system",
    "content": "Nero is a data conversion engine, takes a piece of course json data, and convert `prereq`, `antireq`, `coreq` into json logic rules.",
}

In [1183]:
# Write to jsonlines file
with jsonlines.open("train/train.jsonl", "w") as writer:
    for t in range(NUMBER_OF_EXAMPLES):
        messages = [SYSTEM_MESSAGE]

        messages.append({
            "role": "user",
            "content": json.dumps(ors[t]),
        })

        messages.append({
            "role": "assistant",
            "content": json.dumps(rules[t]),
        })

        l = {
            "messages": messages,
        }

        writer.write(l)
    